In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from parse_input_file import parse_file
from create_feature_dfs import compose_signals_df, create_initial_input, create_df_only_outs

from MA import moving_average
from PACF import PACF

In [ ]:
READ_FROM_FILE_TEST = False
FILE_PATH_FROM_DATA = 'Data/rts1.txt'

In [ ]:
if READ_FROM_FILE_TEST:
    parsing_result = parse_file('Test+')
    
    a_coefs = [parsing_result['out_weights'][i] for i in parsing_result['out_weights'].keys()]
    b_coefs = [parsing_result['manage_weights'][i] for i in parsing_result['manage_weights'].keys()]
    
    features_dfs = compose_signals_df(parsing_result)
elif FILE_PATH_FROM_DATA:
    features_dfs = create_df_only_outs(FILE_PATH_FROM_DATA)
else:
    a_coefs = [0,0.15,-0.33,0.25]
    b_coefs = [1,0.3,0.7,0.]
    
    parsing_result = create_initial_input(a_coefs, b_coefs, uniform_low=0, uniform_high=100, time_series_shape=100, noise_std=1)
    
    features_dfs = compose_signals_df(parsing_result)

In [ ]:
features_dfs.head()

In [ ]:
plt.figure(figsize=(16,16))

features_dfs['''v(k)'''].plot(label='''v(k)''')
features_dfs['''y(k)'''].plot(label='''y(k)''')

plt.legend(loc='upper left')
plt.show()

# Moving average

In [ ]:
moving_avarege_result, w_1 = moving_average(features_dfs['''y(k)'''], n=5)
exponential_ma_result, w_2 = moving_average(features_dfs['''y(k)'''], n=5, weights='exp')
exp_smoothed_ma_result, w_3 = moving_average(features_dfs['''y(k)'''], n=5, weights='exp_smoothed')

In [ ]:
plt.figure(figsize=(16,16))

plt.plot(features_dfs['''y(k)'''], label='initial time series')
plt.plot(range(4,features_dfs.shape[0]),moving_avarege_result, label='Moving avarage')
plt.plot(range(4,features_dfs.shape[0]),exponential_ma_result, label='Exponential Moving avarage')
plt.plot(range(4,features_dfs.shape[0]),exp_smoothed_ma_result, label='Exponential Smoothed Moving avarage')

plt.legend(loc='upper left')
plt.show()

# Patial Auto Correlation Function

In [ ]:
pacf_object = PACF(features_dfs['''y(k)'''])
pacf_results = [pacf_object.simmetric_F(i) for i in range(10)]

In [ ]:
plt.figure(figsize=(16,16))

plt.plot(pacf_results);

In [ ]:
r_results = [pacf_object.count_r(i) for i in range(10)]

plt.figure(figsize=(16,16))

plt.plot(r_results);